# 07 - Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events </br> identification d'évènements 'Defaut', 'Bourrage' et 'Erreur' en criticité ERROR

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [30]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [31]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess_id_events_filtered_07.csv'

# B. Jeux de données

### a) Metrics

In [32]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (2182421, 14)


In [33]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [34]:
metrics_df['timestamp'].min()

'2022-11-02 08:43:45.619000+00:00'

In [35]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

897260


In [36]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-11-02 08:43:46.921000+00:00,INFO,358,iFoil L,iFoil,Gen. 2,Total Pages Counter,25411,User,Operator,ERR,12766,18000
1,iFoil,2022-11-02 08:43:46.921000+00:00,INFO,358,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,670871,User,Operator,ERR,12766,18000
2,PLC,2022-11-02 08:43:46.845000+00:00,ERROR,358,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,3359237,User,Operator,ERR,12766,18000


### b) Jobs

In [37]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (37299, 56)


In [38]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [39]:
jobs_df['started_at'].min()

'2022-02-22 09:43:18.116000+00:00'

In [40]:
jobs_df.head(3)

,total_copies_job,started_at,ended_at,speed_job,paperHeight_job,paperWidth_job,total_copies_requested_job,ifoil_job,scanner_mode,bars_job,...,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies_event,varnishConsumptionVarnish_3d_event
0,6,2022-02-22 09:43:18.116000+00:00,2022-02-22 09:44:33.389000+00:00,313,483,330,6,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,6,4.585923
1,11,2022-02-22 09:45:01.304000+00:00,2022-02-22 09:46:34.929000+00:00,313,483,330,11,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,11,2.917403
2,7,2022-02-22 09:47:30.319000+00:00,2022-02-22 09:48:37.554000+00:00,313,483,330,7,True,3,2,...,0,0,1500,24,25,False,3,SUCCESS,7,0.423666


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [41]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.reset_index()
jobs_reduced.shape

(22186, 56)

## b) Réduction de metrics

### Par criticité ERROR

In [42]:
metrics_reduced = metrics_df[metrics_df['criticality_events'] == 'ERROR']

### Par importance d'identifiant d'évènement

In [43]:
# lignes avec des évènements identifiant une maintenance
id_to_drop_1 = [391, 330, 377, 407, 332, 331, 313, 333, 376, 372, 344, 343, 371, 358, 334, 311, 472, 408, 406, 350, 2, 352, 346, 0]
# lignes avec des évènements identifiant une intervention humaine
id_to_drop_2 = [352, 324, 381, 440, 385, 405, 447, 388, 320, 417, 444, 329, 315, 384, 345, 349, 466, 419]
# lignes avec des évènements identifiant 'arrêt d'urgence'
id_to_drop_3 = [321, 322, 323, 379, 380]
# fusion des listes d'identifiants à supprimer
id_to_drop = id_to_drop_1 + id_to_drop_2 + id_to_drop_3
# on supprime les lignes avec des évènements identifiant une maintenance
metrics_reduced = metrics_reduced[~metrics_reduced['identification_events'].isin(id_to_drop)]
metrics_reduced.shape

(20242, 14)

In [44]:
# on vérifie la distribution par identifiant d'évènements
metrics_reduced['identification_events'].value_counts()

1002    14636
1000     1543
454      1373
355       736
383       534
357       357
445       237
1003      192
354       166
328        92
386        66
356        55
387        51
351        31
465        27
392        24
1001       20
359        18
382        17
389        16
451        15
325        13
479         7
430         6
327         5
446         2
452         1
476         1
418         1
Name: identification_events, dtype: int64

### Par nombre d'identifiant d'évènement

In [45]:
# # Calculer le nombre de valeurs d'identification_events pour chaque groupe
# counts = metrics_by_identification.groupby('identification_events')['identification_events'].transform('count')
# # Filtrer les lignes où le nombre de valeurs d'identification_events est inférieur à 20
# metrics_reduced = metrics_by_identification[counts >= 50]

In [46]:
# # on vérifie la distribution par identification_events
# metrics_reduced['identification_events'].value_counts()

### Premier équilibrage par identifiant d'évènement

In [47]:
# fonction permettant de reduire le nombre de ligne d'une classe à n lignes
def balance_dataframe_by_identification(df, identification, n):
    # on filtre les lignes du df qui ont la valeur criticality dans la colonne 'criticality_events'
    identification_df = df[df['identification_events'] == identification]
    # on selectionne un nombre aléatoire de lignes
    identification_df_sample = identification_df.sample(n=n)
    # on stoke les index des lignes du df qui ne sont pas dans le sample
    index_to_delete = identification_df[~identification_df.index.isin(identification_df_sample.index)].index
    # suppression des lignes exclu du sample
    return df.drop(index_to_delete)

In [48]:
# # on réduit le nombre de classe pour l'id 0
# metrics_reduced = balance_dataframe_by_identification(metrics_reduced,0,2845)
# # on vérifie la distribution par identification_events
# metrics_reduced['identification_events'].value_counts()

In [49]:
# # on vérifie la distribution par criticité
# metrics_reduced['criticality_events'].value_counts()

# D. Création du datatset pour le pre-processing

In [50]:
metrics_reduced['jobId'] = 0
metrics_reduced.shape

(20242, 15)

## a) Analyse des dates

In [51]:
# # Vérifier si des dates communes existent
# dates_communes_exist = jobs_reduced['started_at'].isin(metrics_reduced['timestamp']).any()

# # Afficher le résultat
# if dates_communes_exist:
#     # Filtrer les dates communes
#     dates_communes = jobs_df['started_at'][jobs_df['started_at'].isin(metrics_df['timestamp'])]
    
#     # Compter le nombre de dates communes
#     nombre_dates_communes = len(dates_communes)
    
#     print(f"Des dates communes existent entre les ensembles de données: {nombre_dates_communes}")
# else:
#     print("Aucune date commune n'a été trouvée entre les ensembles de données.")


In [52]:
# création de dictionnaire du nombre de lignes par mois
def compare_datetime_series_shapes(s1, s2):
    s1_serie = {}
    s2_serie = {}
    s1 = pd.to_datetime(s1)
    s2 = pd.to_datetime(s2)
    for i in range(1,13):
        s1_month = s1[s1.dt.month == i]
        s2_month = s2[s2.dt.month == i]
        print(i, s1_month.shape, s2_month.shape)
        s1_serie[i] = s1_month.index
        s2_serie[i] = s2_month.index
    return s1_serie, s2_serie

In [53]:
# on liste les index des lignes par mois
#jobs_indexes_by_month, metrics_indexes_by_month = compare_datetime_series_shapes(jobs_reduced['started_at'], metrics_reduced['timestamp'])

In [54]:
# nombre de lignes par mois
# month = 5

# Selection des lignes de jobs en Mai
# jobs_df['started_at'] = pd.to_datetime(jobs_df['started_at'])
# jobs_batch = jobs_df[ (jobs_df['started_at'].dt.month == month)]
# jobs_batch = jobs_df.loc[jobs_df.index.isin(jobs_indexes_by_month.get(month))]
# jobs_batch.shape

# Selection des lignes de metrics en Mai
# metrics_df['timestamp'] = pd.to_datetime(metrics_df['timestamp'])
# metrics_batch = metrics_df[ (metrics_df['timestamp'].dt.month == month)]
# metrics_batch = metrics_df.loc[metrics_df.index.isin(metrics_indexes_by_month.get(month))]
# metrics_batch.shape

## b) Correspondance de dates entre metrics et jobs

### Pour chaque ligne de metrics quel est le jobId correspondant

In [55]:
import warnings
warnings.simplefilter("ignore")

In [56]:
# DataFrame 1 
# avec les intervalles de temps
dataframe1 = jobs_reduced[['started_at', 'ended_at', 'jobId']]
# conversion des colonnes au format datetime
cols_to_convert = ['started_at', 'ended_at']
dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)
dataframe1 = dataframe1.sort_values('started_at')
dataframe1 = dataframe1.reset_index()
dataframe1.head(3)

,index,started_at,ended_at,jobId
0,15113,2022-11-02 09:44:07.765000+00:00,2022-11-02 09:54:38.330000+00:00,1667382247
1,15114,2022-11-02 09:56:26.583000+00:00,2022-11-02 09:58:18.618000+00:00,1667382986
2,15115,2022-11-02 10:03:09.434000+00:00,2022-11-02 10:04:03.477000+00:00,1667383389


In [57]:
# DataFrame 2 
# avec les valeurs de date
dataframe2 = pd.DataFrame({
    'timestamp': metrics_reduced['timestamp'].values,
    'criticality_events': metrics_reduced['criticality_events'].values,
    'identification_events': metrics_reduced['identification_events'].values,
    'jobId': 0
}, index=None)
# conversion des colonnes au format datetime
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])
dataframe2 = dataframe2.sort_values('timestamp')
dataframe2 = dataframe2.reset_index()
dataframe2.head(3)

,index,timestamp,criticality_events,identification_events,jobId
0,1872,2022-11-02 09:58:13.666000+00:00,ERROR,454,0
1,1873,2022-11-02 10:03:59.444000+00:00,ERROR,454,0
2,1874,2022-11-02 10:08:30.416000+00:00,ERROR,454,0


In [58]:
import time
# début du temps d'execution de la cellule
start = time.time()

# Parcours chaque ligne de job
for index1, row1 in dataframe1.iterrows():
    # creation des variables
    job_starts = row1.started_at.tz_localize(None)
    job_ends = row1.ended_at.tz_localize(None)

    # mois de référence
    month = job_starts.month
    if month != job_ends.month :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')
    else :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')

    for index2, row2 in dataframe2_by_month.iterrows():
        # creation des variables
        metrics_timestamp = row2['timestamp'].tz_localize(None)
        # si le timestamp de la lignes de row2 de metrics est compris dans l'intervalle de temps du job de row1
        if metrics_timestamp >= job_starts :
            if metrics_timestamp <= job_ends :
                dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
    # # avant de passer à la row1 suivante
    # # on affiche l'index1 toutes les 1000 lignes et son temps d'execution
    print(f'[ job index {index1} = time {time.time() - start}]') if index1 % 1000 == 0 and index1 != 0 else None


[ job index 1000 = time 25.989500284194946]
[ job index 2000 = time 52.29332971572876]
[ job index 3000 = time 88.7631139755249]
[ job index 4000 = time 146.02955627441406]
[ job index 5000 = time 213.2057445049286]
[ job index 6000 = time 305.13391876220703]
[ job index 7000 = time 418.49250507354736]
[ job index 8000 = time 658.1225395202637]
[ job index 9000 = time 785.2871351242065]
[ job index 10000 = time 893.6047794818878]
[ job index 11000 = time 1025.301515340805]
[ job index 12000 = time 1207.563359260559]
[ job index 13000 = time 1321.693449497223]
[ job index 14000 = time 1361.7484011650085]
[ job index 15000 = time 1395.8393666744232]
[ job index 16000 = time 1417.6741116046906]
[ job index 17000 = time 1435.5716753005981]
[ job index 18000 = time 1460.852697134018]
[ job index 19000 = time 1482.1419050693512]
[ job index 20000 = time 1502.663051366806]
[ job index 21000 = time 1520.6600284576416]
[ job index 22000 = time 1540.1073954105377]


In [59]:
# # Prétraitement : Tri et conversion de timezone
# dataframe1['started_at'] = dataframe1['started_at'].dt.tz_localize(None)
# dataframe1['ended_at'] = dataframe1['ended_at'].dt.tz_localize(None)
# dataframe2['timestamp'] = dataframe2['timestamp'].dt.tz_localize(None)
# dataframe2 = dataframe2.sort_values('timestamp')

# # Optimisation du traitement
# for index1, row1 in dataframe1.iterrows():
#     job_starts, job_ends = row1['started_at'], row1['ended_at']
#     month = job_starts.month

#     # Filtrage conditionnel
#     condition = ((dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month)) & (dataframe2['jobId'] == 0)
#     dataframe2.loc[condition & (dataframe2['timestamp'] >= job_starts) & (dataframe2['timestamp'] <= job_ends), 'jobId'] = row1['jobId']

#     # Affichage pour le suivi
#     if index1 % 1000 == 0 and index1 != 0:
#         print(f'[ job index {index1} = time {time.time() - start}]')

In [60]:
# on vérifie la distribution par identification_events pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
dataframe2[dataframe2['jobId'] != 0]['identification_events'].value_counts()

454     1306
355      401
357      232
1000      81
1002      78
354       71
1003      52
356       50
383       19
451       14
359       11
386        8
387        3
325        3
328        2
445        1
351        1
446        1
382        1
476        1
Name: identification_events, dtype: int64

In [61]:
print(f'Nombre de ligne de metrics : {dataframe2.shape[0]} (dont {dataframe2[dataframe2.jobId != 0].shape[0]} avec un jobId associé)')

Nombre de ligne de metrics : 20242 (dont 2336 avec un jobId associé)


## c) Fusion des dataframes jobs et metrics

In [62]:
merged_df = jobs_reduced.merge(dataframe2, on='jobId')
merged_df.shape

(2336, 60)

In [63]:
# on vérifie la distribution par criticité
merged_df['identification_events'].value_counts()

454     1306
355      401
357      232
1000      81
1002      78
354       71
1003      52
356       50
383       19
451       14
359       11
386        8
387        3
325        3
328        2
445        1
351        1
446        1
382        1
476        1
Name: identification_events, dtype: int64

In [64]:
# on vérifie la distribution par criticité
merged_df['criticality_events'].value_counts()

ERROR    2336
Name: criticality_events, dtype: int64

### Dataframe ne contenant que des criticality = ERROR

In [65]:
# merged_df_error = merged_df[merged_df.criticality_events == 'ERROR']

In [66]:
# on vérifie la distribution par criticité
# merged_df_error['identification_events'].value_counts()

### Equilibrage des classes identification_events en fonction de l'id le plus fréquent des lignes ERROR

In [67]:
# # on identifie la classe identification_events la plus fréquente dans les ERROR
# identification_counts = merged_df[merged_df['criticality_events'] == 'ERROR']['identification_events'].value_counts()
# most_frequent_identification_event = identification_counts.idxmax()
# nombre_lignes = identification_counts[most_frequent_identification_event]
# print(f"L'id '{most_frequent_identification_event}' est le plus fréquent des lignes ERROR avec {nombre_lignes} lignes.")

## d) Output

In [68]:
merged_df.to_csv(path_or_buf=save_csv)